<h1> Capstone Project - The Battle of Neighborhoods </h1>

## Select the Location for a New Chinese Restaurant in Singapore

### Introduction
---
Due to the on-going COVID19 pandemic, restaurants are closed for dining and people are staying at home. This is the time when people start to miss the past days when they could eat out and have some get-togethers with friends and families.

With the wish for everything back to normal very soon, I choose topic number 2 and will try to find a good location for a Chinese restaurant in Singapore.

In this notebook, I will document the process of selecting a location for a new Chinese Restaurant. The Foursquare API will be called to gather data for various places in Singapore.

In [2]:
import pandas as pd
import json
import numpy as np # library to handle data in a vectorized manner
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium
import geocoder
import time
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

First I try to explore 200m around where I live

In [24]:
address = 'Bedok'
geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Bedok are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bedok are 1.3239765, 103.930216.


In [25]:
CLIENT_ID = '1MKQYPD5YAFP0RLBQ0DUGJ1SVG0T0NLV1BLMPNLMFJ1OVKDG' 
CLIENT_SECRET = 'T42UXBPJKL0XROLMHAR5GQK0U2L3QCRTTXBICWQA25410ARC' 
VERSION = '20200505'
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)
            
        # make the GET request
results = requests.get(url).json()

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [63]:
neighborhood = results['response']['headerLocation']
neighborhood

'Bedok'

In [64]:
venues = results['response']['groups'][0]['items']
neighborhood = results['response']['headerLocation']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['Neighborhood'] = neighborhood
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/ruiping/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,Neighborhood
0,Ya Kun Kaya Toast 亞坤,Coffee Shop,1.324095,103.929198,Bedok
1,Song Zhou Luo Bo Gao 松洲箩卜糕,Breakfast Spot,1.324836,103.930520,Bedok
2,Bedok Chwee Kueh 勿洛水粿,Chinese Restaurant,1.324903,103.930250,Bedok
3,Duke Bakery,Bakery,1.324691,103.932514,Bedok
4,FairPrice Fínest,Supermarket,1.324140,103.929260,Bedok


In [70]:
# create map of Bedok using latitude and longitude values
map_bedok = folium.Map(location=[latitude, longitude], zoom_start=15)
# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bedok) 
map_bedok

From the map it is clear that most of te venues are near the Bedok MRT staiton, so we will continue to explore all MRT stations in Singapore.

### About the Data

The list of MRT stations in Singapore is available on [wikipedia](https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations), and it contains all MRT stations including stations being built or planned to be built. The stations I am interested in are the MRT stations which are currently in operation, and are interchange stations.
The geography coordinates are not included in that table, and they can be obtained by using **geocoder** with all the station names searched.
With the station names and their coordinates, we can explore the interchange stations by calling **foursqure** API and get the most visited venues for each station. Finally we apply *kmeans clustering* and use **folium** to visualise the popular venue distributions among interchange stations and select a area for a new Chinese restaurant.

### Data pre-processing
---
In this part all Singapore MRT names and locations will be obtained by an easy web scraping using **pandas.read_html**, then I will extract the interchange MRT stations, search their coordinates, finally **Foursquare** API will be called to get the most visted venues for all interchange MRT stations in Singapore, and a dataframe will be created with all the information.

First I got a list of MRT stations from wikipedia

In [3]:
mrt = pd.read_html('https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations', header=0)
df = mrt[2]

This table has some merged headers, I will re-shape it

In [4]:
df = df.iloc[1:,]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 1 to 254
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Alpha-numeric code(s)             185 non-null    object
 1   Alpha-numeric code(s).1           129 non-null    object
 2   Station name                      254 non-null    object
 3   Station name.1                    253 non-null    object
 4   Station name.2                    235 non-null    object
 5   Opening                           254 non-null    object
 6   Name(s) during planning stages    254 non-null    object
 7   Abbreviation                      254 non-null    object
 8   Location(s)                       254 non-null    object
 9   Connection(s) to other transport  81 non-null     object
dtypes: object(10)
memory usage: 20.0+ KB


In [5]:
df

,Alpha-numeric code(s),Alpha-numeric code(s).1,Station name,Station name.1,Station name.2,Opening,Name(s) during planning stages,Abbreviation,Location(s),Connection(s) to other transport
1,North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL)
2,NS1 EW24,JE5,Jurong East,裕廊东,ஜூரோங் கிழக்கு,10 March 1990,Jurong East,JUR,Jurong East,Jurong East Temporary Bus Interchange
3,NS2,NaN,Bukit Batok,武吉巴督,புக்கிட் பாத்தோக்,10 March 1990,Bukit Batok South,BBT,Bukit Batok,Bukit Batok Bus Interchange
4,NS3,NaN,Bukit Gombak,武吉甘柏,புக்கிட் கோம்பாக்,10 March 1990,Bukit Batok North,BGB,Bukit Batok,NaN
5,NaN,NS3A,Brickland,红砖,பிரிக்லேன்ட்,Mid-2030s,Brickland,TBA,Exact location not yet known,NaN
...,...,...,...,...,...,...,...,...,...,...
250,NaN,CR13 TE7,Bright Hill,光明山,பிரைட் ஹில்,2029,Bright Hill,BRH,Bishan,NaN
251,Punggol Extension (CRLe),Punggol Extension (CRLe),Punggol Extension (CRLe),Punggol Extension (CRLe),Punggol Extension (CRLe),Punggol Extension (CRLe),Punggol Extension (CRLe),Punggol Extension (CRLe),Punggol Extension (CRLe),Punggol Extension (CRLe)
252,NaN,CP2,Elias,伊莱雅,இலியாஸ்,2031,Elias,TBA,Pasir Ris,NaN
253,PE4,CP3,Riviera,里维拉,ரிவியாரா,2031,Riviera,TBA,Punggol,NaN


All those "NaN" under the first column are future stations which are not in operation, they should be dropped

In [6]:
# simply drop whole row with NaN in "Alpha-numeric code(s)" column
df.dropna(subset=["Alpha-numeric code(s)"], axis=0, inplace=True)
# reset index, because we droped some rows
df.reset_index(drop=True, inplace=True)

/Users/ruiping/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df.head()

,Alpha-numeric code(s),Alpha-numeric code(s).1,Station name,Station name.1,Station name.2,Opening,Name(s) during planning stages,Abbreviation,Location(s),Connection(s) to other transport
0,North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL),North South Line (NSL)
1,NS1 EW24,JE5,Jurong East,裕廊东,ஜூரோங் கிழக்கு,10 March 1990,Jurong East,JUR,Jurong East,Jurong East Temporary Bus Interchange
2,NS2,NaN,Bukit Batok,武吉巴督,புக்கிட் பாத்தோக்,10 March 1990,Bukit Batok South,BBT,Bukit Batok,Bukit Batok Bus Interchange
3,NS3,NaN,Bukit Gombak,武吉甘柏,புக்கிட் கோம்பாக்,10 March 1990,Bukit Batok North,BGB,Bukit Batok,NaN
4,NS4 BP1,JS1,Choa Chu Kang,蔡厝港,சுவா சூ காங்,10 March 1990,Choa Chu Kang,CCK,Choa Chu Kang,Choa Chu Kang Bus Interchange


I also need to drop the rows which are actually sub headers from the source, after inspection, they are rows contains parentheses and numbers, a "dot" symbol which is Unicode Decimal Code 8226(2022 in hex) and a cell called "Infill station"

And reset index after dropping those rows

In [8]:
df_mrt = df.loc[df['Station name'].str.contains(r"\(.*\)|[0-9]|\u2022") == False]
df_mrt = df_mrt.loc[df_mrt['Station name'].str.contains('Infill station') == False]
df_mrt.reset_index(drop=True, inplace=True)
df_mrt.head()

,Alpha-numeric code(s),Alpha-numeric code(s).1,Station name,Station name.1,Station name.2,Opening,Name(s) during planning stages,Abbreviation,Location(s),Connection(s) to other transport
0,NS1 EW24,JE5,Jurong East,裕廊东,ஜூரோங் கிழக்கு,10 March 1990,Jurong East,JUR,Jurong East,Jurong East Temporary Bus Interchange
1,NS2,NaN,Bukit Batok,武吉巴督,புக்கிட் பாத்தோக்,10 March 1990,Bukit Batok South,BBT,Bukit Batok,Bukit Batok Bus Interchange
2,NS3,NaN,Bukit Gombak,武吉甘柏,புக்கிட் கோம்பாக்,10 March 1990,Bukit Batok North,BGB,Bukit Batok,NaN
3,NS4 BP1,JS1,Choa Chu Kang,蔡厝港,சுவா சூ காங்,10 March 1990,Choa Chu Kang,CCK,Choa Chu Kang,Choa Chu Kang Bus Interchange
4,NS5,NaN,Yew Tee,油池,இயூ டீ,10 February 1996,Choa Chu Kang North,YWT,Choa Chu Kang,NaN


Select useful information: *Alpha-numeric code(s), Station name(in Malay or English), Station name(in Chinese), Location(s)*

In [9]:
header = ['Alpha-numeric code(s)', 'Station name', 'Station name.1', 'Location(s)']
df_mrt = df_mrt[header]

In [10]:
df_mrt.head()

,Alpha-numeric code(s),Station name,Station name.1,Location(s)
0,NS1 EW24,Jurong East,裕廊东,Jurong East
1,NS2,Bukit Batok,武吉巴督,Bukit Batok
2,NS3,Bukit Gombak,武吉甘柏,Bukit Batok
3,NS4 BP1,Choa Chu Kang,蔡厝港,Choa Chu Kang
4,NS5,Yew Tee,油池,Choa Chu Kang


It is better to rename *Alpha-numeric code(s)* as *Code*, *Station name.1* as *Station name in Chinese* and *Location(s)* to *Location*

In [11]:
df_mrt = df_mrt.rename(columns={'Alpha-numeric code(s)':'Code','Station name.1': 'Station name in Chinese', 'Location(s)': 'Location'})
df_mrt.head()

,Code,Station name,Station name in Chinese,Location
0,NS1 EW24,Jurong East,裕廊东,Jurong East
1,NS2,Bukit Batok,武吉巴督,Bukit Batok
2,NS3,Bukit Gombak,武吉甘柏,Bukit Batok
3,NS4 BP1,Choa Chu Kang,蔡厝港,Choa Chu Kang
4,NS5,Yew Tee,油池,Choa Chu Kang


Inspect all MRT station names, looking good

In [195]:
station_name = df_mrt['Station name']
for station in station_name:
    print(station)

Jurong East
Bukit Batok
Bukit Gombak
Choa Chu Kang
Yew Tee
Kranji
Marsiling
Woodlands
Admiralty
Sembawang
Canberra
Yishun
Khatib
Yio Chu Kang
Ang Mo Kio
Bishan
Braddell
Toa Payoh
Novena
Newton
Orchard
Somerset
Dhoby Ghaut
City Hall
Raffles Place
Marina Bay
Marina South Pier
Pasir Ris
Tampines
Simei
Tanah Merah
Bedok
Kembangan
Eunos
Paya Lebar
Aljunied
Kallang
Lavender
Bugis
City Hall
Raffles Place
Tanjong Pagar
Outram Park
Tiong Bahru
Redhill
Queenstown
Commonwealth
Buona Vista
Dover
Clementi
Jurong East
Chinese Garden
Lakeside
Boon Lay
Pioneer
Joo Koon
Gul Circle
Tuas Crescent
Tuas West Road
Tuas Link
Expo
Changi Airport
HarbourFront
Outram Park
Chinatown
Clarke Quay
Dhoby Ghaut
Little India
Farrer Park
Boon Keng
Potong Pasir
Woodleigh
Serangoon
Kovan
Hougang
Buangkok
Sengkang
Punggol
Dhoby Ghaut
Bras Basah
Esplanade
Promenade
Nicoll Highway
Stadium
Mountbatten
Dakota
Paya Lebar
MacPherson
Tai Seng
Bartley
Serangoon
Lorong Chuan
Bishan
Marymount
Caldecott
Farrer Road
Holland Village
B

#### Get interchange stations
Then I will look for the interchange stations, which are rows with more than 1 elements in **Code** column

This will make the following search faster and I can focus our data analysis on more crowded region

In [12]:
interchange_mask = (df_mrt['Code'].str.len() > 4)
df_interchange = df_mrt.loc[interchange_mask]
df_interchange.head()

,Code,Station name,Station name in Chinese,Location
0,NS1 EW24,Jurong East,裕廊东,Jurong East
3,NS4 BP1,Choa Chu Kang,蔡厝港,Choa Chu Kang
7,NS9 TE2,Woodlands,兀兰,Woodlands
15,NS17 CC15,Bishan,碧山,Bishan
19,NS21 DT11,Newton,纽顿,Newton


After that we need to remove the **Code** column and drop duplicated rows since the original table is listed by MRT lines, and interchange stations appear more than once, which gives duplicated entries

In [13]:
df_interchange=df_interchange.drop('Code',1)
df_interchange=df_interchange.drop_duplicates()
df_interchange.reset_index(drop=True, inplace=True)

With all the unique Station name, we can find their coordinates using **Nominatim**

In [17]:
station_name = df_interchange['Station name']

station_list = []
geolocator = Nominatim(user_agent="sgp_explorer")
for address in station_name: 
    location = geolocator.geocode(address+',Singapore')
    print("{}: {}, {}".format(address, location.latitude, location.longitude))
    # Slow down request frequency to avoid timeout error
    time.sleep(5)
    station_list.append([address,location.latitude, location.longitude])
    

Jurong East: 1.333115, 103.7422968
Choa Chu Kang: 1.3847493, 103.7445341
Woodlands: 1.436897, 103.786216
Bishan: 1.3509859, 103.84825507492937
Newton: 1.31318325, 103.83804024301281
Dhoby Ghaut: 1.29935345, 103.84530856672068
City Hall: 1.2930272, 103.85264337981533
Raffles Place: 1.2835416999999998, 103.85146023266938
Marina Bay: 1.2755589999999999, 103.85489740049655
Tampines: 1.3546528, 103.9435712
Tanah Merah: 1.3272541, 103.9465046
Paya Lebar: 1.31821785, 103.89323462550945
Bugis: 1.2999532999999999, 103.85527814689878
Outram Park: 1.2806497, 103.8402787
Buona Vista: 1.3070851, 103.79057908744186
Expo: 1.3354428, 103.9620852
HarbourFront: 1.2653951, 103.8224032
Chinatown: 1.2837372, 103.8437976
Little India: 1.3066476, 103.8492691
Serangoon: 1.3498624, 103.8737292
Sengkang: 1.3916536000000002, 103.89536361089395
Punggol: 1.4052585, 103.9023302
Promenade: 1.2927082, 103.8611244
MacPherson: 1.3267681, 103.8903019
Bayfront: 1.2825724, 103.8597242
Bukit Panjang: 1.378629, 103.7621358


Now we can create a new dataframe

In [18]:
interchange_coordinates = pd.DataFrame(station_list)
interchange_coordinates.columns = ['Station name', 'Latitude', 'Longitude']
interchange_coordinates

,Station name,Latitude,Longitude
0,Jurong East,1.333115,103.742297
1,Choa Chu Kang,1.384749,103.744534
2,Woodlands,1.436897,103.786216
3,Bishan,1.350986,103.848255
4,Newton,1.313183,103.838040
5,Dhoby Ghaut,1.299353,103.845309
6,City Hall,1.293027,103.852643
7,Raffles Place,1.283542,103.851460
8,Marina Bay,1.275559,103.854897
9,Tampines,1.354653,103.943571


Merge 2 dataframes so that we can have a new dataframe with all MRT interchange stations' coordinates

And the data pre-processing is done, we now have the data for all MRT interchange stations with the station names in Chinese, the location and the coordinates.

The data can be used to explore each MRT station using **Foursqure** API

In [22]:
interchange_merged = df_interchange
interchange_merged = pd.merge(interchange_merged, interchange_coordinates,on='Station name',how='left')
#interchange_merged = interchange_merged.join(interchange_coordinates.set_index('Station name'), on='Station name')
interchange_merged=interchange_merged.drop_duplicates()
interchange_merged.reset_index(drop=True, inplace=True)
interchange_merged

,Station name,Station name in Chinese,Location,Latitude,Longitude
0,Jurong East,裕廊东,Jurong East,1.333115,103.742297
1,Choa Chu Kang,蔡厝港,Choa Chu Kang,1.384749,103.744534
2,Woodlands,兀兰,Woodlands,1.436897,103.786216
3,Bishan,碧山,Bishan,1.350986,103.848255
4,Newton,纽顿,Newton,1.313183,103.838040
5,Dhoby Ghaut,多美歌,Museum Planning Area,1.299353,103.845309
6,City Hall,政府大厦,Downtown Core,1.293027,103.852643
7,Raffles Place,莱佛士坊,Downtown Core,1.283542,103.851460
8,Marina Bay,滨海湾,"Downtown Core, Straits View Planning Area",1.275559,103.854897
9,Tampines,淡滨尼,Tampines,1.354653,103.943571


#### Exploring each MRT interchange station
Make API calls and data frame *singapore_venues* is created to contain the data in *interchange_merged* and venues

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        responses = requests.get(url).json()["response"]
        #districts = responses["headerLocation"]
        results = responses['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Station', 
                  'Station Latitude', 
                  'Station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
singapore_venues = getNearbyVenues(names=interchange_merged['Station name'],
                                   latitudes=interchange_merged['Latitude'],
                                   longitudes=interchange_merged['Longitude']
                                  )

Group by stations and count venues for each station

In [29]:
singapore_venues.groupby('Station').count()

,Station Latitude,Station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Station,,,,,,
Bayfront,45,45,45,45,45,45
Bishan,45,45,45,45,45,45
Bugis,100,100,100,100,100,100
Bukit Panjang,32,32,32,32,32,32
Buona Vista,46,46,46,46,46,46
Chinatown,100,100,100,100,100,100
Choa Chu Kang,23,23,23,23,23,23
City Hall,61,61,61,61,61,61
Dhoby Ghaut,77,77,77,77,77,77


In [30]:
print('There are {} uniques categories among those MRT stations.'.format(len(singapore_venues['Venue Category'].unique())))

There are 219 uniques categories among those MRT stations.


### Clustering and Data Analysis

Start with onehot encoding to handle categorical data

In [31]:
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")
# add Station column back to dataframe, it will appear as the last column
singapore_onehot['Station'] = singapore_venues['Station']
# move Station column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

singapore_onehot.head()

,Station,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Jurong East,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Jurong East,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Jurong East,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Jurong East,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Jurong East,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by station by taking the mean of the frequency of occurrence of each category

In [32]:
singapore_grouped = singapore_onehot.groupby('Station').mean().reset_index()
singapore_grouped.head()

,Station,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Bayfront,0.022222,0.00000,0.0,0.0,0.022222,0.022222,0.00,0.000000,0.0,...,0.00,0.0,0.000000,0.0,0.044444,0.0,0.000000,0.0,0.00,0.00
1,Bishan,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.00,0.022222,0.0,...,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.00
2,Bugis,0.000000,0.00000,0.0,0.0,0.020000,0.010000,0.01,0.010000,0.0,...,0.01,0.0,0.020000,0.0,0.000000,0.0,0.010000,0.0,0.01,0.01
3,Bukit Panjang,0.000000,0.03125,0.0,0.0,0.000000,0.000000,0.00,0.062500,0.0,...,0.00,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00,0.00
4,Buona Vista,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.00,0.021739,0.0,...,0.00,0.0,0.021739,0.0,0.000000,0.0,0.021739,0.0,0.00,0.00


Create another dataframe with the 10 most visited venues for each station

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
station_venues_sorted = pd.DataFrame(columns=columns)
station_venues_sorted['Station'] = singapore_grouped['Station']

for ind in np.arange(singapore_grouped.shape[0]):
    station_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

station_venues_sorted.head()

,Station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayfront,Boutique,Scenic Lookout,Garden,Hotel,Waterfront,Roof Deck,Lounge,Bridge,Accessories Store,Park
1,Bishan,Coffee Shop,Food Court,Bubble Tea Shop,Cosmetics Shop,Pet Store,Café,Japanese Restaurant,Chinese Restaurant,Ice Cream Shop,Supermarket
2,Bugis,Bakery,Café,Hotel,Cocktail Bar,Dessert Shop,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Coffee Shop,Sandwich Place
3,Bukit Panjang,Fast Food Restaurant,Coffee Shop,Shopping Mall,Noodle House,Sushi Restaurant,Asian Restaurant,Café,Fried Chicken Joint,Gym,Supermarket
4,Buona Vista,Japanese Restaurant,Indian Restaurant,Food Court,Chinese Restaurant,Shopping Mall,Café,Bakery,Dessert Shop,Coffee Shop,Performing Arts Venue


In [35]:
station_venues_sorted.to_csv('station_venues_sorted.csv')

Run clustering for 5 clusters(we expect to have clusters for east, west, center, south and north)

In [43]:
# set number of clusters
kclusters = 5

singapore_grouped_clustering = singapore_grouped.drop('Station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 0, 0, 0, 0, 3, 3, 0], dtype=int32)

Create a new dataframe with clustering labels, station names, station Chinese names, station coordinates, and 1-10 most common venues

In [38]:
# add clustering labels
station_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

singapore_stations = interchange_merged

# merge with the dataframe created in the data pre-processing part
singapore_stations = singapore_stations.merge(station_venues_sorted, left_on='Station name', right_on='Station',how='left')
singapore_stations=singapore_stations.drop_duplicates()
singapore_stations.reset_index(drop=True, inplace=True)
#Drop the Station column, it is the same as Station name
singapore_stations=singapore_stations.drop('Station',1)


singapore_stations

,Station name,Station name in Chinese,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Jurong East,裕廊东,Jurong East,1.333115,103.742297,0,Chinese Restaurant,Coffee Shop,Food Court,Café,Japanese Restaurant,Shopping Mall,Steakhouse,Multiplex,Bubble Tea Shop,Clothing Store
1,Choa Chu Kang,蔡厝港,Choa Chu Kang,1.384749,103.744534,0,Coffee Shop,Food Court,Portuguese Restaurant,Thai Restaurant,Playground,Fast Food Restaurant,Sandwich Place,Chinese Restaurant,Food Truck,Café
2,Woodlands,兀兰,Woodlands,1.436897,103.786216,0,Café,Coffee Shop,Japanese Restaurant,Shopping Mall,Asian Restaurant,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Electronics Store,Clothing Store
3,Bishan,碧山,Bishan,1.350986,103.848255,0,Coffee Shop,Food Court,Bubble Tea Shop,Cosmetics Shop,Pet Store,Café,Japanese Restaurant,Chinese Restaurant,Ice Cream Shop,Supermarket
4,Newton,纽顿,Newton,1.313183,103.838040,0,Chinese Restaurant,Seafood Restaurant,Italian Restaurant,Hotel Bar,Hotel,Convenience Store,Grocery Store,Gym / Fitness Center,Noodle House,Thai Restaurant
5,Dhoby Ghaut,多美歌,Museum Planning Area,1.299353,103.845309,3,Hotel,Café,Park,Japanese Restaurant,Cosmetics Shop,History Museum,Bubble Tea Shop,Theater,Hobby Shop,Karaoke Bar
6,City Hall,政府大厦,Downtown Core,1.293027,103.852643,3,Hotel,Shopping Mall,Japanese Restaurant,Coffee Shop,French Restaurant,Steakhouse,Event Space,Concert Hall,Bookstore,Cocktail Bar
7,Raffles Place,莱佛士坊,Downtown Core,1.283542,103.851460,3,Hotel,Café,Food Court,Gym,Cocktail Bar,Coffee Shop,Sandwich Place,Salad Place,Waterfront,Shopping Mall
8,Marina Bay,滨海湾,"Downtown Core, Straits View Planning Area",1.275559,103.854897,4,Yoga Studio,Harbor / Marina,Spanish Restaurant,Plaza,Building,Gastropub,Mexican Restaurant,Government Building,Seafood Restaurant,Bus Line
9,Tampines,淡滨尼,Tampines,1.354653,103.943571,0,Bakery,Café,Coffee Shop,Fast Food Restaurant,Supermarket,Gym,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Japanese Restaurant


In [39]:
singapore_stations.to_csv('singapore_stations.csv')

We can see that Chinese Restaurant is the top most common venues for many MRT stations, and they are all under Cluster 0.


In [40]:
chinese_restaurants = singapore_stations[singapore_stations['1st Most Common Venue'] == 'Chinese Restaurant'].reset_index(drop=True)
chinese_restaurants

,Station name,Station name in Chinese,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Jurong East,裕廊东,Jurong East,1.333115,103.742297,0,Chinese Restaurant,Coffee Shop,Food Court,Café,Japanese Restaurant,Shopping Mall,Steakhouse,Multiplex,Bubble Tea Shop,Clothing Store
1,Newton,纽顿,Newton,1.313183,103.838040,0,Chinese Restaurant,Seafood Restaurant,Italian Restaurant,Hotel Bar,Hotel,Convenience Store,Grocery Store,Gym / Fitness Center,Noodle House,Thai Restaurant
2,HarbourFront,港湾,Bukit Merah,1.265395,103.822403,0,Chinese Restaurant,Japanese Restaurant,Fast Food Restaurant,Toy / Game Store,Clothing Store,Multiplex,Coffee Shop,Bakery,Malay Restaurant,Noodle House
3,Chinatown,牛车水,Outram,1.283737,103.843798,0,Chinese Restaurant,Food Court,Hostel,Vegetarian / Vegan Restaurant,Italian Restaurant,Spa,Café,French Restaurant,Japanese Restaurant,Beer Garden


In [41]:
chinese_restaurants.to_csv('chinese_restaurants.csv')

Clustering results visualisation

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(singapore_stations['Latitude'], singapore_stations['Longitude'], singapore_stations['Station name'], singapore_stations['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Discussion
---
From the last dataframe *chinese_restaurant.csv*, we can see that **Jurong East** is a place people go shopping and dining, and on the map it is far from the city area, so it should be a very populated residential area(which is true!).

Cluster 3 is the city area, where venues are more close to one another. This is the place where the most tourist attractions are. It is also noticed that **Chinatown**, where Chinese Restaurant is the top venue is also located near that area.

It might be apparant to say that Chinatown should be selected as the new Chinese Restaurant location, and it is indeed a pace where many popular Chinese Restaurants are located. However, city area costs more investments, and from the *chinese_restaurants.csv* we can see that a Chinese Restaurant in Chinatown tends to have more competations not only from other Chinese restaurants, but also from food courts and other Asian Restaurants. 

Therefore I would apply *the Blue Ocean Strategy* and select **Jurong East** as the location to build a neighborhood Chinese Restaurant. **Jurong East** MRT interchange is a residential area with many shopping venues, Chinese Restaurants are popular there, and it is far from the compitations

### Conclusion
---
In this project, MRT stations in Singapre are gathered from [wikipedia](https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations) using **pandas.read_html**, all interchange stations are extracted and their coordinates are found. With those information, **Foursquare** API is called to explore 200ms around all the MRT interchange stations in Singapore, and their top 10 most common venues are found. Then new data frame is being created by filtering the MRT station where Chinese Restaurant is the top common venue. Finally clustering was applied on the MRT data all data is visualised on a map plotted by **folium**. With all the data analysed and visualised, a conclusion was drawn that the new Chinese Restaurant is to be located around **Jurong East** station.